# 4일차 2교시 기본 연산 다루기

### 목차
* 1. 기본 연산자
* 2. 데이터 스키마
* 3. 데이터프레임
* 4. 참고자료
  * https://spark.apache.org/docs/latest/api/java/index.html?org/apache/spark/sql/types/StructField.html
  * https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations

### 1. 기본 연산자

#### 1-1. Transformation & Lazy Evaluation
+ RDD(Rsilient Distributed Dataset): 불변성을 가지며 병렬로 처리할 수 있는 파티셔닝된 레코드의 모음
    + RDD: 불변성, 한번 생성하면 변경할 수 없음
+ Transformation: 원하는 변경 방법을 알려주는 명령. 추상적인 변경 방법이며, 실제 수행X
+ Lazy Evaluation(지연 연산): 특정 연산 명령을 하면 즉시 데이터를 수정하지 않고, 원시 데이터에 적용할 트랜스포메이션 실행계획 생성하고, 실제로 연산 그래프를 처리하기 직전까지 기다림
+ 전체 데이터 흐름 최적화

---
#### 1-2. DataFrame
| 함수 | 설명 | 기타 |
| - | - | - |
| df.printSchema() | 스키마 정보를 출력합니다. | - |
| df.schema | StructType 스키마를 반환합니다 | - |
| df.columns | 컬럼명 정보를 반환합니다 | - |
| df.show(n) | 데이터 n 개를 출력합니다 | - |
| df.first() | 데이터 프레임의 첫 번째 Row 를 반환합니다 | - |
| df.head(n) | 데이터 프레임의 처음부터 n 개의 Row 를 반환합니다 | - |
| df.createOrReplaceTempView | 임시 뷰 테이블을 생성합니다 | - |
| df.sample(withReplacement, fraction, seed) | 랜덤 샘플링을 수행합니다 (복원여부, 비율, 시드) | 시드 지정을 통해 일정한 샘플링 결과를 얻을 수 있습니다 |
| df.randomSplit() | 랜덤 스플릿을 수행합니다 | - |
| df.union(newdf) | 데이터프레임 간의 유니온 연산을 수행합니다 | - |
| df.limit(n) | 추출할 로우수 제한 | T |
| df.repartition(n) | 파티션 재분배, 셔플발생 | - |
| df.coalesce() | 셔플하지 않고 파티션을 병합 | 마지막 스테이지의 reduce 수가 줄어드는 효과로 성능저하에 유의해야 합니다 |
| df.collect() | 모든 데이터 수집, 반환 | A |
| df.take(n) | 상위 n개 로우 반환 | A |
| df.toLocalIterator() | iterator로 모든 파티션의 데이터를 드라이버에 전달, 파티션을 차례대로 반복처리 | - |

---
#### 1-3. DataFrame 컬럼관련 메서드
| 함수 | 설명 | 기타 |
| - | - | - |
| df.select | 컬럼이나 표현식 사용  | - |
| df.selectExpr | 문자열 표현식 사용 = df.select(expr()) | - |
| df.withColumn(컬럼명, 표현식) | 컬럼 추가, 비교, 컬럼명 변경 | - |
| df.withColumnRenamed(old_name, new_name) | 컬럼명 변경 | - |
| df.drop() | 컬럼 삭제 | - |
| df.where | 로우 필터링 | - |
| df.filter | 로우 필터링 | - |
| df.sort, df.orderBy | 정렬 | - |
| df.sortWithinPartitions | 파티션별 정렬 | - |

---
#### 1-4. 컬럼관련 함수
| 함수 | 설명 | 기타 |
| - | - | - |
| expr("someCol - 5") | 표현식 | - |
| lit() | 리터럴 | - |
| cast() | 컬럼 데이터 타입 변경 | - |
| distinct() | unique row | - |
| desc(), asc() | 정렬 순서 | - |

---
#### 1-5. 로우
| 함수 | 설명 | 기타 |
| - | - | - |
| Row() | 로우 생성 | - |


#### 1-6 Structured API 활용 가이드

+ <strong>DataFrame</strong> 구성요소: 레코드, 컬럼
    + 레코드: Row타입(Table의 로우)
    + 컬럼: 레코드에 수행할 연산 표현식(테이블의 컬럼)
+ <strong>스키마</strong>: 각 컬럼명과 데이터 타입을 정의
+ <strong>파티셔닝</strong>: DataFrame이나 Dataset이 클러스터에서 물리적으로 배치되는 형태를 정의
+ <strong>파티셔닝 스키마</strong>: 파이션을 배치하는 방법을 정의

In [16]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Data Engineer Intermediate Day4") \
    .config("spark.dataengineer.intermediate.day4", "tutorial-2") \
    .getOrCreate()

In [18]:
""" 스키마 확인하기 """
df = spark.read.format("json").load("./data/flight-data/json/2015-summary.json") # 미국 교통통계국이 제공하는 항공운항 데이터
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [19]:
df2 = spark.read.load("./data/flight-data/json/2015-summary.json", format="json")
df2.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [20]:
df3 = spark.read.json("./data/flight-data/json/2015-summary.json")
df3.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



> 위에서 사용한 3가지 모두 동일한 결과를 얻을 수 있으며 편한 방식의 구조화된 API 를 활용하시면 됩니다.

### 2. 데이터 스키마

#### 2.1 직접 스키마를 지정하여 읽는 방법
+ 스키마: 여러 개의 StructField 타입 필드로 구성된 StructType 객체
+ StructField: (이름), (데이터 타입), (컬럼이 값이 없거나 null일 수 있는지 지정하는 불리언 값) 으로 구성
---
+ https://spark.apache.org/docs/latest/api/java/index.html?org/apache/spark/sql/types/StructField.html
+ StructField(String name, DataType dataType, boolean nullable, Metadata metadata)
+ Metadata: 해당 컬럼과 관련된 정보이며, 스파크의 머신러닝 라이브러리에서 사용

In [21]:
""" 스키마를 직접 만들어 적용 """
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
    StructField("DEST_COUNTRY_NAME", StringType(), True),
    StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
    StructField("count", LongType(), False, metadata={"hello":"world"})
])

df = spark.read.format("json")\
    .schema(myManualSchema)\
    .load("data/flight-data/json/2015-summary.json")

In [23]:
df.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [24]:
df2 = spark.read.load("data/flight-data/json/2015-summary.json", format="json", schema=myManualSchema)
df.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



> 위에서 사용한 2가지 모두 동일한 결과를 얻을 수 있으며 편한 방식의 구조화된 API 를 활용하시면 됩니다.

#### 2.2 컬럼
+ <strong>컬럼: 표현식을 사용해 레코드 단위로 계산한 값을 단순하게 나타내는 논리적인 구조</strong> (테이블의 컬럼으로 생각할 수 있음)
+ 컬럼의 실제값을 얻으려면 로우가 필요하고, 로우를 얻으려면 DataFrame이 필요 (로우는 데이터 레코드)
+ DataFrame을 통하지 않으면 외부에서 컬럼에 접근 불가
+ 즉, DF -> row -> column
---
+ 컬럼의 내용을 수정하려면 반드시 DataFrame의 스파크 트랜포메이션을 사용
+ col, column 함수를 사용하는 것이 가장 간편 (책에서는 col 함수 사용)
+ 컬럼은 컬럼명을 카탈로그에 저장된 정보와 비교하기 전까지 미확인 상태
+ 분석기가 동작하는 단계에서 컬럼과 테이블을 분석

In [25]:
""" 컬럼 생성 """
from pyspark.sql.functions import col, column

col("someColumnName")

Column<b'someColumnName'>

#### 2.3 표현식
+ <strong>표현식: DataFrame 레코드의 여러 값에 대한 트랜스포메이션 집합을 의미</strong>
+ 여러 컬럼명을 입력받아 식별하고 단일 값을 만들기 위해 다양한 표현식을 각 레코드에 적용하는 함수
+ <strong>표현식은 expr함수로 사용</strong>
+ <strong>컬럼은 단지 표현식일 뿐</strong>
+ <strong>표현식은 연산 순서를 지정하는 논리적 트리로 컴파일됨</strong>

In [26]:
from pyspark.sql.functions import expr

print(expr("someCol - 5"))
print(expr("someCol") - 5)
print(col("someCol") - 5)

# 동일한 트랜스포메이션 과정: 스파크는 연산 순서를 지정하는 논리적 트리로 컴파일합니다
(((col("somecol") + 5) * 200) - 6) < col("othercol")

Column<b'(someCol - 5)'>
Column<b'(someCol - 5)'>
Column<b'(someCol - 5)'>


Column<b'((((somecol + 5) * 200) - 6) < othercol)'>

In [27]:
""" 논리적 트리로 컴파일되는 표현식 """
from pyspark.sql.functions import expr
expr("(((someCol + 5) * 200) - 6) < otherCol")

Column<b'((((someCol + 5) * 200) - 6) < otherCol)'>

![dag.png](image/dag.png)

In [28]:
""" columns 속성을 사용 """
spark.read.format("json").load("./data/flight-data/json/2015-summary.json").columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

#### 2.4 레코드와 로우
+ 스파크는 레코드를 Row 객체로 표현
    + (각 로우는 하나의 레코드, '로우'와 '레코드'를 같은 의미로 사용)
    + (대문자로 시작하는 Row는 Row 객체를 의미)
+ Row 객체는 내부에 바이트 배열을 가지며, 오직 컬럼 표현식으로만 다룰 수 있으므로 사용자에게 노출되지 않음
+ DataFrame을 사용해 드라이버에게 개별 로우를 반환하는 명령은 항상 하나 이상의 Row 타입을 반환
+ Row 객체는 스키마 정보를 가지고 있지 않음 (DataFrame만 유일하게 스키마를 가지고 있음)
+ Row 객체를 직접 생성하려면 DataFrame의 스키마와 같은 순서로 값을 명시

In [29]:
""" Row를 확인하는 예문 """
df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

In [30]:
df.head(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1)]

In [43]:
""" 로우 생성하기 """
from pyspark.sql import Row
myRow = Row("Hello", None, 1, False)

""" 로우 접근하기 """
print(myRow[0])
print(myRow[2])
[*myRow]

Hello
1


['Hello', None, 1, False]

### 3 데이터프레임

![transform](image/transform.png)

+ 로우나 컬럼 추가
+ 로우나 컬럼 제거
+ 로우를 컬럼으로 변환하거나, 그 반대로 변환
+ 컬럼값을 기준으로 로우 순서 변경

#### 3.1 DataFrame 생성하기
+ 원시 데이터소스에서 DataFrame을 생성하고 임시 뷰를 등록
+ Row 객체를 가진 Seq 타입을 직접 전환하여 DataFrame을 생성



In [45]:
""" 원시 데이터소스 활용 """
df = spark.read.format("json").load("data/flight-data/json/2015-summary.json")
df.createOrReplaceTempView("2015_summary")

sql_result = spark.sql("SELECT * FROM 2015_summary").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [46]:
""" Row 객체 활용 """
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
    StructField("some", StringType(), True),
    StructField("col", StringType(), True),
    StructField("names", LongType(), False)
])

myRow = Row("Hello", None, 1)
myDf = spark.createDataFrame([myRow], myManualSchema)
myDf.show(1)

+-----+----+-----+
| some| col|names|
+-----+----+-----+
|Hello|null|    1|
+-----+----+-----+



#### 3.2 select와 selectExpr

In [47]:
""" 단일 혹은 다중 컬럼 설정 """
df.select("DEST_COUNTRY_NAME").show(2)
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



In [18]:
""" 컬럼을 참조하는 다앙한 방법 """
df.select(
    "DEST_COUNTRY_NAME",
    expr("DEST_COUNTRY_NAME"),
    col("DEST_COUNTRY_NAME"),
    column("DEST_COUNTRY_NAME")
).show(2)

+-----------------+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|    United States|
|    United States|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+-----------------+
only showing top 2 rows



In [48]:
""" expr를 이용한 컬럼 참조 """
df.select(expr("DEST_COUNTRY_NAME AS destination")).show(2)
df.select(expr("DEST_COUNTRY_NAME AS destination").alias("DEST_COUNTRY_NAME")).show(2)

+-------------+
|  destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



+ select메서드에 expr 함수를 사용하는 패턴을 자주 사용함
+ 스파크는 이를 위해 selectExpr 메서드를 제공

In [49]:
""" selectExpr 활용 예문 """
df.selectExpr("DEST_COUNTRY_NAME as newColmnName", "DEST_COUNTRY_NAME").show(2)
df.selectExpr("*", "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry").show(2)

+-------------+-----------------+
| newColmnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [50]:
""" 집계함수 지정하기 """
df.selectExpr("avg(count) as averageCount", "count(distinct(DEST_COUNTRY_NAME)) as distinctCount").show()

+------------+-------------+
|averageCount|distinctCount|
+------------+-------------+
| 1770.765625|          132|
+------------+-------------+



#### 3.3 스파크 데이터 타입으로 변환하기

In [51]:
""" 
리터럴(literal)을 사용한 컬럼 추가
리터럴: 어떤 상수나 프로그래밍으로 생성된 변수값이 특정 컬럼의 값보다 큰지 확인할 때 사용
"""
from pyspark.sql.functions import lit

df.select(expr("*"), lit(1).alias("One")).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



#### 3.4 컬럼 추가하기


In [52]:
""" 
withColumn으로 컬럼 추가 
withColumn(컬럼명, 표현식)
"""
df.withColumn("numberOne", lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [53]:
""" 컬럼 비교 """
df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME")).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [57]:
""" 컬럼명 바꾸기 """
before = df
before.printSchema()

after = before.withColumn("Destination", expr("DEST_COUNTRY_NAME"))
after.printSchema()
after.drop("Destination").columns

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)
 |-- Destination: string (nullable = true)



['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

#### 3.5 컬럼명 바꾸기

In [58]:
""" withColumnRenamed """
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").columns

['dest', 'ORIGIN_COUNTRY_NAME', 'count']

#### 3.6 예약 문자와 키워드
+ 공백이나 하이픈(-) 같은 예약 문자를 컬럼명에 사용하려면 백틱(`) 문자를 사용해야 함

In [59]:
""" withColumn, selectExpr, select 차이점 """
dfWithLongColName = df.withColumn("This Long Column-Name", expr("ORIGIN_COUNTRY_NAME")) # 첫 번째 인수에서 사용하지 않음
dfWithLongColName.show(2)

dfWithLongColName.selectExpr("`This Long Column-Name`", "`This Long Column-Name` as `new col`").show(2) # 사용함
dfWithLongColName.select(expr("`This Long Column-Name`")).show(2)

+-----------------+-------------------+-----+---------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|This Long Column-Name|
+-----------------+-------------------+-----+---------------------+
|    United States|            Romania|   15|              Romania|
|    United States|            Croatia|    1|              Croatia|
+-----------------+-------------------+-----+---------------------+
only showing top 2 rows

+---------------------+-------+
|This Long Column-Name|new col|
+---------------------+-------+
|              Romania|Romania|
|              Croatia|Croatia|
+---------------------+-------+
only showing top 2 rows

+---------------------+
|This Long Column-Name|
+---------------------+
|              Romania|
|              Croatia|
+---------------------+
only showing top 2 rows



#### 3.7 대소문자 구분
+ 기본적으로 스파크는 대소문자를 가리지 않음
```
set spark.sql.caseSensitive true # 대소문자를 구분하기 위한 옵션
```

#### 3.8 컬럼 제거하기

In [29]:
""" drop 함수 """

df.drop("ORIGIN_COUNTRY_NAME").columns

['DEST_COUNTRY_NAME', 'count']

In [62]:
df.printSchema()

spark.conf.set('spark.sql.caseSensitive', True)
caseSensitive = df.drop("dest_country_name")
caseSensitive.printSchema()

spark.conf.set('spark.sql.caseSensitive', False)
caseInsensitive = df.drop("dest_country_name")
caseInsensitive.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

root
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [63]:
""" drop 함수 """

dfWithLongColName.drop("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").columns # 여러 컬럼을 지우기

['count', 'This Long Column-Name']

#### 3.9 컬럼의 데이터 타입 변경하기

In [72]:
""" cast 함수 """
df.printSchema()

int2str = df.withColumn("str_count", col("count").cast("string"))
int2str.show(5)
int2str.printSchema()

str2int = int2str.withColumn("int_count", col("str_count").cast("int"))
str2int.show(5)
str2int.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|str_count|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|       15|
|    United States|            Croatia|    1|        1|
|    United States|            Ireland|  344|      344|
|            Egypt|      United States|   15|       15|
|    United States|              India|   62|       62|
+-----------------+-------------------+-----+---------+
only showing top 5 rows

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)
 |-- str_count: string (nullable = true)

+-----------------+-------------------+-----+---------+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|str_count|int_count|
+---------------

#### 3.10 로우 필터링하기

In [74]:
""" filter, where 함수 """
df.where(col("count") < 2).show(2)
df.filter(col("count") < 2).show(2)

df.where("count < 2").show(2)
df.filter("count < 2").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+----

In [75]:
""" 같은 표현식에 여러 필터를 적용 """
df.where(col("count") < 2).where(col("ORIGIN_COUNTRY_NAME") != "Croatia").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



#### 3.11 고유한 로우 얻기

In [76]:
""" distinct 함수 """
print(df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count())
print(df.select("ORIGIN_COUNTRY_NAME").distinct().count())
# distinctcount?

256
125


#### 3.12 무작위 샘플 만들기

In [79]:
""" sample 함수 """
seed = 5
withReplacement = False
fraction = 0.5
df.sample(withReplacement, fraction, seed).count()

126

#### 3.13 임의 분할하기

In [81]:
""" randomSplit 함수 """
seed = 42
dataFrames = df.randomSplit([0.25, 0.75], seed)
print(dataFrames[0].count())
print(dataFrames[1].count())

66
190


#### 3.14 로우 합치기와 추가하기
+ 동일한 스키마와 컬럼 수를 가져야 함

In [82]:
""" union 함수 """
from pyspark.sql import Row

schema = df.schema
newRows = [
    Row("New Country", "Other Country", 5),
    Row("New Country 2", "Other Country 3", 1)
]

# Parallelized Collections :
# Parallelized collections are created by calling SparkContext’s parallelize method on an existing iterable or collection in your driver program.
# The elements of the collection are copied to form a distributed dataset that can be operated on in parallel. 

parallelizedRows = spark.sparkContext.parallelize(newRows) 
newDF = spark.createDataFrame(parallelizedRows, schema)

newDF.show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|      New Country|      Other Country|    5|
|    New Country 2|    Other Country 3|    1|
+-----------------+-------------------+-----+



In [84]:
df.union(newDF)\
    .where("count = 1")\
    .where(col("ORIGIN_COUNTRY_NAME") != "United States")\
    .show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|    United States|          Gibraltar|    1|
|    United States|             Cyprus|    1|
|    United States|            Estonia|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



#### 3.15 로우 정렬하기
+ asc, desc 함수를 사용하여 정렬 순서를 지정
+ asc_nulls_first, desc_nulls_first, asc_nulls_last, desc_nulls_last 메서드로 null의 정렬 순서를 지정
+ sortWithinPartitions 함수는 파티션별 정렬을 지원 (트랜스포메이션 처리 전 성능 최적화 목적)

In [85]:
""" sort, orderBy 함수 """
df.sort("count").show(5)
df.orderBy("count", "DEST_COUNTRY_NAME").show(5)
df.orderBy(col("count"), col("DEST_COUNTRY_NAME")).show(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--

In [86]:
""" 정렬순서 지정하기 """
from pyspark.sql.functions import desc, asc

df.orderBy(expr("count desc")).show(5)
df.orderBy(col("ORIGIN_COUNTRY_NAME").desc(), col("DEST_COUNTRY_NAME").asc()).show(5) # 예제 변경

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Vietnam|    2|
|    United States|          Venezuela|  246|
|    United States|            Uruguay|   13|
|          Algeria|      United States|    4|
|           Angola|      United States|   15|
+-----------------+-------------------+-----+
only showing top 5 rows



In [97]:
""" 파티션별 정렬 """ 
# 최적화는 내일 세션에서 소개할 예정
sortedWithPartitions = spark.read.format("json").load("data/flight-data/json/*-summary.json").sortWithinPartitions("count")
normalDataframe = spark.read.format("json").load("data/flight-data/json/*-summary.json").sort("count")

In [98]:
sortedWithPartitions.explain(True)
# print(sortedWithPartitions.rdd.getNumPartitions())

normalDataframe.explain(True)
# print(normalDataframe.rdd.getNumPartitions())

== Parsed Logical Plan ==
'Sort ['count ASC NULLS FIRST], false
+- Relation[DEST_COUNTRY_NAME#1100,ORIGIN_COUNTRY_NAME#1101,count#1102L] json

== Analyzed Logical Plan ==
DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint
Sort [count#1102L ASC NULLS FIRST], false
+- Relation[DEST_COUNTRY_NAME#1100,ORIGIN_COUNTRY_NAME#1101,count#1102L] json

== Optimized Logical Plan ==
Sort [count#1102L ASC NULLS FIRST], false
+- Relation[DEST_COUNTRY_NAME#1100,ORIGIN_COUNTRY_NAME#1101,count#1102L] json

== Physical Plan ==
*(1) Sort [count#1102L ASC NULLS FIRST], false, 0
+- *(1) FileScan json [DEST_COUNTRY_NAME#1100,ORIGIN_COUNTRY_NAME#1101,count#1102L] Batched: false, Format: JSON, Location: InMemoryFileIndex[file:/home/jovyan/work/data/flight-data/json/2011-summary.json, file:/home/jovy..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:bigint>
== Parsed Logical Plan ==
'Sort ['count ASC NULLS FIRST], true
+- 

#### 3.16 로우 수 제한하기


In [99]:
df.limit(5).show()
df.orderBy(expr("count desc")).limit(6).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
+--------------------+-------------------+-----+



#### 3.17 repartition과 cocalesce
+ 향후에 사용할 파티션 수가 현재 파티션 수보다 많거나 컬럼을 기준으로 파티션을 만드는 경우에 사용(repartition, 셔플이 필수로 발생)
+ 자주 필터링되는 컬럼을 기준으로 파티션 재분배를 권장
+ repartition: 물리적인 데이터 구성 제어

In [104]:
""" 파티션 나누기 """
df.rdd.getNumPartitions()
repart_1 = df.repartition(5)
repart_2 = df.repartition(col("DEST_COUNTRY_NAME"))
repart_3 = df.repartition(5, col("DEST_COUNTRY_NAME"))

# repartition 숫자 변경 등 실험내용 공유
print(repart_1.rdd.getNumPartitions())
print(repart_2.rdd.getNumPartitions())
print(repart_3.rdd.getNumPartitions())

5
200
5


+ 셔플하지 않고 파티션을 병합

In [105]:
""" 파티션 합치기 """
df.repartition(5, col("DEST_COUNTRY_NAME")).coalesce(2)

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

#### 3.18 드라이버로 로우 데이터 수집하기
+ 대규모 데이터셋에 collect 명령을 수행하면 드라이버 비정상 종료 우려

In [106]:
collectDF = df.limit(5)
collectDF.take(5) # 정수를 인수값으로 사용

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [107]:
collectDF.show()  # 결과를 정돈된 형태로 출력

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+



In [108]:
collectDF.show(5, False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
+-----------------+-------------------+-----+



In [109]:
collectDF.collect() # 전체 모든 테이터를 수집, 반환

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [110]:
collectDF.toLocalIterator()

+ 대규모 데이터셋에 collect, toLocalIterator 수행하면 매우 큰 비용(cpu, 메모리, 네트워크) 발생, 드라이버 비정상적 종료 가능성

### cast

In [111]:
""" Row 객체 활용 """
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
    StructField("some", StringType(), True),
    StructField("num", StringType(), True),
    StructField("count", StringType(), True)
])

myRow1 = Row("Hello", 1, 1)
myRow2 = Row("Hello", None, 'aaa')
myDf = spark.createDataFrame([myRow1, myRow2], myManualSchema)
myDf.show()

+-----+----+-----+
| some| num|count|
+-----+----+-----+
|Hello|   1|    1|
|Hello|null|  aaa|
+-----+----+-----+



In [112]:
myDf.withColumn("num2", col("num").cast("integer")).show()

+-----+----+-----+----+
| some| num|count|num2|
+-----+----+-----+----+
|Hello|   1|    1|   1|
|Hello|null|  aaa|null|
+-----+----+-----+----+



In [113]:
myDf.withColumn("count2", col("count").cast("integer")).show()

+-----+----+-----+------+
| some| num|count|count2|
+-----+----+-----+------+
|Hello|   1|    1|     1|
|Hello|null|  aaa|  null|
+-----+----+-----+------+



### distinct count

In [114]:
df.selectExpr("count(distinct(ORIGIN_COUNTRY_NAME))").show()

+-----------------------------------+
|count(DISTINCT ORIGIN_COUNTRY_NAME)|
+-----------------------------------+
|                                125|
+-----------------------------------+



In [115]:
print(df.select("ORIGIN_COUNTRY_NAME").distinct().count())

125


In [116]:
from pyspark.sql.functions import countDistinct, approxCountDistinct

df.select(countDistinct("ORIGIN_COUNTRY_NAME")).show()

+-----------------------------------+
|count(DISTINCT ORIGIN_COUNTRY_NAME)|
+-----------------------------------+
|                                125|
+-----------------------------------+



https://spark.apache.org/docs/2.2.1/api/java/org/apache/spark/sql/functions.html#approx_count_distinct-org.apache.spark.sql.Column-double-

In [117]:
df.select(approxCountDistinct("ORIGIN_COUNTRY_NAME", rsd=0.05)).show()
# rsd: Maximum estimation error allowed

+------------------------------------------+
|approx_count_distinct(ORIGIN_COUNTRY_NAME)|
+------------------------------------------+
|                                       116|
+------------------------------------------+



In [118]:
df.select(approxCountDistinct("ORIGIN_COUNTRY_NAME", rsd=0.1)).show()
# rsd: Maximum estimation error allowed

+------------------------------------------+
|approx_count_distinct(ORIGIN_COUNTRY_NAME)|
+------------------------------------------+
|                                       101|
+------------------------------------------+

